<p>
<font size='10' face='Georgia, Arial'>IIC2115 - Programación como Herramienta para la Ingeniería</font><br>
</p>

## Ejemplos de consultas en SQL

Considere la base de datos contenida en el archivo `data.db`, formada por las tablas `Flights`, `Aircraft`, `Employees` y `Certified`, que fueron creadas utilizando las siguientes sentencias en SQL:
* `CREATE TABLE Flights(flno INTEGER PRIMARY KEY, origin TEXT, destination TEXT, distance INTEGER, departs DATE, arrives DATE, price REAL)`
* `CREATE TABLE Aircraft(aid INTEGER PRIMARY KEY, aname TEXT, cruisingrange INTEGER)`
* `CREATE TABLE Employees(eid INTEGER PRIMARY KEY, ename TEXT, salary REAL)`
* `CREATE TABLE Certified(eid INTEGER, aid INTEGER, PRIMARY KEY(eid,aid), FOREIGN KEY(eid) REFERENCES employees, FOREIGN KEY(aid) REFERENCES aircraft)`


Considere que la tabla `Employees` describe tanto a pilotos como a otro tipo de trabajador, que cada piloto está certificado para volar al menos un avión, y que solo los pilotos están certificados para volar.

En base a esto, y asumiendo que las tablas ya están correctamente pobladas, escriba las consultas SQL para responder a las preguntas que se indican a continuación. Considere que cada pregunta debe ser respondida utilizando una única consulta en SQL.

- Encuentre los nombres de los pilotos certificados para volar algún avión Boeing.
- Encuentre los nombres de los pilotos que están certificados para volar aviones con autonomía de más de 1000 millas.
- Por cada piloto que está certificado para volar más de tres aviones, encuentre el _id_ y la autonomía del avión con máxima autonomía para el que está certificado.
- Encuentre los _id_ de los aviones que pueden ser usados en rutas de Los Ángeles a Chicago.
- Encuentre los nombres de los pilotos cuyo sueldo es menor que el precio del vuelo más barato de Los Ángeles a Honolulu.
- Encuentre los nombres de los aviones cuyos pilotos certificados tienen todos sueldos mayores a US$ 80.000.

### Exploración de los datos
Antes de comenzar a escribir las consultas, revisaremos el contenido de cada tabla:

In [ ]:
import sqlite3

In [ ]:
connection = sqlite3.connect('data.db')
cursor = connection.cursor()

In [ ]:
sql = "SELECT * FROM Flights"
cursor.execute(sql)
for r in cursor:
    print(r)

In [ ]:
sql = "SELECT * FROM Aircraft"
cursor.execute(sql)
for r in cursor:
    print(r)

In [ ]:
sql = "SELECT * FROM Employees"
cursor.execute(sql)
for r in cursor:
    print(r)

In [ ]:
sql = "SELECT * FROM Certified"
cursor.execute(sql)
for r in cursor:
    print(r)

In [ ]:
connection.close()

### Consultas

In [ ]:
# Encuentre los nombres de los pilotos certificados para volar algún avión Boeing.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT E.ename \
                FROM Employees E, Certified C, Aircraft A \
                WHERE E.eid = C.eid AND C.aid = A.aid AND A.aname LIKE '%Boeing%'")
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Encuentre los nombres de los pilotos que están certificados para volar aviones con autonomía de más de 1000 millas.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute('SELECT DISTINCT E.ename \
                FROM Employees E, Certified C, Aircraft A \
                WHERE C.aid = A.aid AND E.eid = C.eid AND A.cruisingrange > 1000')
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Encuentre los nombres de los pilotos que están certificados para volar SOLO aviones con autonomía de más de 1000 millas.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute('SELECT E.ename \
                FROM Employees E, Certified C, Aircraft A \
                WHERE C.aid = A.aid AND E.eid = C.eid \
                GROUP BY E.eid, E.ename \
                HAVING MIN(A.cruisingrange) > 1000')
for r in cursor:
    print(r[0])
connection.close()

In [ ]:
# Por cada piloto que está certificado para volar más de tres aviones, encuentre el id y nombre del piloto, junto con la autonomía del avión
# con máxima autonomía para el que está certificado.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute('SELECT E.eid, E.ename, MAX (A.cruisingrange) \
                FROM Employees E, Certified C, Aircraft A \
                WHERE C.aid = A.aid AND E.eid = C.eid\
                GROUP BY C.eid \
                HAVING COUNT (*) > 3')
for r in cursor:
    print(r[0], r[1], r[2])
connection.close()

In [ ]:
# Encuentre los id y nombres de los aviones que pueden ser usados en rutas de Los Ángeles a Chicago.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT A.aid, A.aname \
                FROM Aircraft A \
                WHERE A.cruisingrange > (SELECT MIN (F.distance) \
                                         FROM Flights F \
                                         WHERE F.origin = 'Los Angeles' AND F.destination = 'Chicago')")
for r in cursor:
    print(r[0], r[1])
connection.close()

In [ ]:
# Encuentre los nombres de los pilotos cuyo sueldo es menor que el precio del vuelo más barato de Los Ángeles a Honolulu.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT E.ename \
                FROM Employees E \
                WHERE E.salary < (SELECT MIN (F.price) \
                                  FROM Flights F \
                                  WHERE F.origin = 'Los Angeles' AND F.destination = 'Honolulu')")
for r in cursor:
    print(r[0])
connection.close()


In [ ]:
# Encuentre los nombres de los aviones cuyos pilotos certificados tienen todos sueldos mayores a US$ 80.000.

connection = sqlite3.connect('data.db')
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT A.aname \
                FROM Aircraft A \
                WHERE A.Aid IN (SELECT C.aid \
                                FROM Certified C, Employees E \
                                WHERE C.eid = E.eid AND NOT EXISTS (SELECT * \
                                                                    FROM Employees E1 \
                                                                    WHERE E1.eid = E.eid AND E1.salary < 80000))")
for r in cursor:
    print(r[0])
connection.close()